In [49]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate
import figure 
from EconModel import cpptools

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)


plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings

do_compile = True
threads = 20

#cpptools.setup_nlopt(folder='cppfuncs/', do_print = True) #install nlopt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Solve the model

In [50]:

specs = {}
T = 20



#specs.update({f'model_NC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'threads':threads,'bargaining':2}}})
#specs.update({f'model_FC':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'threads':threads,'bargaining':0}}})
#specs.update({f'model_LC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK':False , 'threads':threads,'bargaining':1}}})
specs.update({f'test':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'threads':threads,'bargaining':0}}})

#specs.update({f'model_FC_doHK':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': True, 'threads':threads,'bargaining':0}}})
#specs.update({f'model_LC_doHK':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': True , 'threads':threads,'bargaining':1}}}) #ser ikke godt ud!
#specs.update({f'model_NC_doHK':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': True, 'threads':threads,'bargaining':2}}})

print(specs)



# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    print(f'{name} loading...',end='')
    
    # setup model
    models = LimitedCommitmentModelClass(name=name,par=spec['par'])
    models.spec = spec

    compile_now = True if do_compile & (m==0) else False
    models.link_to_cpp(force_compile=compile_now)
    
    print(' solving...')
    %time models.solve() 
    
    #Save the data 
    #T = models.par.T
    #print(' saving...')
    #models.sim.init_love[:] = 0.2
    #models.sim.init_A[:] = 2.0 
    #np.random.seed(models.par.seed)
    #data1 = Estimate.create_data(models,start_p = 1, end_p = T-1, to_xl = True, name_xl = name)
    #print(f'Couple {np.mean(models.sim.couple,0)}')
    #print(f'HKw {np.mean(models.sim.Kw,0)}')
    #print(f'HKm {np.mean(models.sim.Km,0)}')
    #print(f'Laborw {np.mean(models.sim.labor_w,0)}')
    #print(f'Laborm {np.mean(models.sim.labor_m,0)}')
    #print(f'Asset {np.nanmean(models.sim.A,0)}')

    #var_list = ('labor_w', 'labor_m', 'cons_w', 'cons_m', 'Vw', 'Vm')
    #t_list = (1,9)
    #i_A = 250
    #i_HK = 1
    #i_HKw = i_HK
    #i_HKm = i_HK
    #i_Z = 0
    #i_Zw = i_Z
    #i_Zm = i_Z
    #i_P = 5
    #i_L = 5
    #figure.figure_single(models, var_list, t_list, i_A, i_Z, i_HK, model_name = name)
    #figure.figure_couple(models, var_list, t_list, i_A, i_Zw, i_Zm, i_HKw, i_HKm, i_L, i_P, sol_type = 'couple', model_name = name)
    #figure.figure_couple(models, var_list, t_list, i_A, i_Zw, i_Zm, i_HKw, i_HKm, i_L, i_P, sol_type = 'remain_couple', model_name = name)



    



#model = models['model FC, NO_HK ']
#model.sim.init_love[:] =0.2
##model.simulate()
#print('HK')*
#print(f'Couple {np.mean(model.sim.couple,0)}')

#print(f'Laborw {np.mean(model.sim.labor_w,0)}')
#print(f'Laborm {np.mean(model.sim.labor_m,0)}')
#print(f'Consumption W {np.mean(model.sim.cons_w,0)}')
#print(f'Consumption M {np.mean(model.sim.cons_m,0)}')
#print(f'Asset {np.nanmean(model.sim.A,0)}')
#print(f'HKw {np.mean(model.sim.Kw,0)}')
#print(f'HKm {np.mean(model.sim.Km,0)}')
#temp = model.sim.power
#I = model.sim.couple<1
#nan = np.zeros(I.shape)
#nan[I] = np.nan
#temp = np.nanmean(temp + nan,axis=0)
#print(f'Power {temp}')


{'test': {'latexname': 'limited', 'par': {'T': 20, 'do_HK': False, 'threads': 20, 'bargaining': 0}}}
test loading...

 solving...
CPU times: total: 8min 54s
Wall time: 52.1 s


In [51]:
stop
T = models.par.T
print(' saving...')
models.sim.init_love[:] = 0.2
    
models.sim.init_A[:] = 2.0 
np.random.seed(models.par.seed)
data1 = Estimate.create_data(models,start_p = 1, end_p = T-1, to_xl = True, name_xl = name)
print(f'Couple {np.mean(models.sim.couple,0)}')
print(f'HKw {np.mean(models.sim.Kw,0)}')
print(f'HKm {np.mean(models.sim.Km,0)}')
print(f'Laborw {np.mean(models.sim.labor_w,0)}')
print(f'Laborm {np.mean(models.sim.labor_m,0)}')
print(f'Asset {np.nanmean(models.sim.A,0)}')


model = models

sol = models.sol
par = models.par
sim =  models.sim

print(f'HKw max {np.max(model.sim.Kw,0)}')
print(f'HKm max {np.max(model.sim.Km,0)}')


print(f'HKw min {np.min(model.sim.Kw,0)}')
print(f'HKm min {np.min(model.sim.Km,0)}')
#print(sim.draw_Kw_temp)
print(sim.draw_Kw)

print(f'A max {np.nanmax(model.sim.A,0)}')
print(f'A min {np.nanmin(model.sim.A,0)}')
print(sim.draw_Kw_temp)

print(f'Love max {np.nanmax(model.sim.love,0)}')
print(f'Love mean {np.nanmean(model.sim.love,0)}')
print(f'Love min {np.nanmin(model.sim.love ,0)}')




print(f'laborW max {np.nanmax(model.sim.labor_w,0)}')
print(f'laborW mean {np.nanmean(model.sim.labor_w,0)}')
print(f'LaborW min {np.nanmin(model.sim.labor_w,0)}')

print(par.grid_A)

NameError: name 'stop' is not defined

In [52]:
do_estimate_omega = True
models.sim.init_love[:] = 0.2
models.sim.init_A[:] = 2.0
np.random.seed(models.par.seed)

data = Estimate.create_data(models,start_p = 1, end_p = models.par.T-1)
data = Estimate.create_variable(data, models.par)
data, wald_FC , wald_NC  = Estimate.main_est(data, do_estimate_omega = True, print_reg = True)

#TODO: 
    # CHEK AT DEN GIVER DET SAMME SOM STATA 
    # wald_FC, wald_NC ud til flere kørninger

c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shadow_value[i] = pd.qcut(data_regress[i], 10, labels = False, duplicates='raise')
c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shadow_value[i] = pd.qcut(data_regress[i], 10, labels = False, duplicates='raise')
c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:298: SettingWithCopyWarning: 
A value is trying to be set on a c

                            OLS Regression Results                            
Dep. Variable:                    y_w   R-squared:                       0.882
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     745.8
Date:                Wed, 20 Mar 2024   Prob (F-statistic):               0.00
Time:                        14:25:31   Log-Likelihood:                 81540.
No. Observations:               38504   AIC:                        -1.624e+05
Df Residuals:                   38161   BIC:                        -1.595e+05
Df Model:                         342                                         
Covariance Type:              cluster                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0304    

In [ ]:
#print(data.shape)
#print(data.info())
#print(data.head()) #preriew the head, god måde at checke om den gør det rigtige
# data = data[data.couple>0.5] #select row that meet certain critereia
#data = data[data['couple']>0.5]
#data = data.drop(columns = ['Love','util','value']) #drop unnesecary columns
#data = data.rename(columns= {'wealth': 'wealth_lag'})
#data.isnull().sum() #sum up all null values for all columns, som algorithm does not like it 
#data = data.fillna(500) Replace all null values with 500
#data[data.wealth == 0 ].head() #printer hvor dette gælder
# data = data.dropna() #drop nan! 
#sum(data.duplicated(data.columns)) #check number of duplicates
#data.describe() #desciptive statistics
#data['district'] = data['district'].replace([1,2,3,4,5],['M'.'B','C', 'Q','S'])
# pd.get_dummies(data['t'], columns = ['t'], prefix = 'd) #convert categorical to dummy
# sample = data.sample(n=10) # a random sample of 10 homes


NameError: name 'stop' is not defined

In [ ]:
#MAIN ESTIMATE  HERTIL LAV DENNE MAIN, FIND UD AF HVORDAN DEN SKAL SE UD?
print_reg = True 

X_t=pd.get_dummies(data[['t', 'init_barg']], columns = ['t','init_barg'], prefix = ['D_t','D_init_barg'], dtype = float) 
#X_t=pd.get_dummies(data['t''init_barg'], columns = ['t'], prefix = 'D_t', dtype = float, drop_first=True,  ) 
X_t = X_t.drop(columns = ['D_t_1','D_init_barg_1']) #drop reference cat





    #drop nan 
    #print(data_regress)

if do_estimate_omega:
    data['wage_shock']=data['omega_res_w']
    data['wage_shock_l']=data['omega_res_w_l']
    data['wage_shock_l2']=data['omega_res_w_l2']
    data['wage_shock_j']=data['omega_res_m']
    data['wage_shock_j_l']=data['omega_res_m_l']
    data['wage_shock_j_l2']=data['omega_res_m_l2']
else:
    data['wage_shock']=data['delta_omega_w']
    data['wage_shock_l']=data['delta_omega_w_l']
    data['wage_shock_l2']=data['delta_omega_w_l2']
    data['wage_shock_j']=data['delta_omega_m']
    data['wage_shock_j_l']=data['delta_omega_m_l']
    data['wage_shock_j_l2']=data['delta_omega_m_l2']

data['BMI'] = data['delta_log_BMI_w']
data['BMI_l'] = data['delta_log_BMI_w_l']
data['BMI_l2'] = data['delta_log_BMI_w_l2']
data['BMI_j'] = data['delta_log_BMI_m']
data['BMI_j_l'] = data['delta_log_BMI_m_l']
data['BMI_j_l2'] = data['delta_log_BMI_m_l2']

data_regress = data[['y_w', 'idx', 'wage_shock','wage_shock_l','wage_shock_l2','wage_shock_j','wage_shock_j_l','wage_shock_j_l2','BMI','BMI_l','BMI_l2','BMI_j','BMI_j_l','BMI_j_l2','control_part_inc_w','control_cons','delta_log_wealth','delta_log_wealth_l','delta_log_wealth_l2']]

#DROP NAN
data_regress = data_regress.dropna() # det ser ud som om den ikke fjerner nogen
import statsmodels.api as sm

    #Run main regression
    #x = data[['X3','X4','X5','X6','X7a','X7b','X8a','X8b','X9','X10','X11','X12','X13','X14','X15']] #with Z

y  = data_regress['y_w']
x = pd.concat([data_regress.drop(columns = ['y_w', 'idx']),  X_t ], axis=1 )
x = sm.add_constant(x) 

#MANGLER 

    # -SKYGGEVÆRDI AF WEALTH ,

    #  Kigge på standardfejl
    # - teste at det virker



result2 = sm.OLS(y,x).fit()
result = sm.OLS(y,x).fit().get_robustcov_results(cov_type = 'cluster', groups = data_regress['idx'])  #TODO: use correct standard errors

if print_reg:
    print('Result, w')
    print(result.summary())

    print('Result,cluster ')
    print(result2.summary())

Wald_FC = result.wald_test('(wage_shock_l=0, wage_shock_l2=0,wage_shock_l=0,wage_shock_j_l=0, wage_shock_j_l2=0, BMI=0, BMI_l=0, BMI_l2=0,BMI_l=0,BMI_j_l=0, BMI_j_l2=0,D_init_barg_0=0,D_init_barg_2=0)', use_f = False)
print(Wald_FC )


Result, w
                            OLS Regression Results                            
Dep. Variable:                    y_w   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.480
Method:                 Least Squares   F-statistic:                     425.1
Date:                Wed, 20 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:24:45   Log-Likelihood:                 89570.
No. Observations:               89314   AIC:                        -1.791e+05
Df Residuals:                   89283   BIC:                        -1.788e+05
Df Model:                          30                                         
Covariance Type:              cluster                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   

c:\Users\czk481\Anaconda3\lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 13, but rank is 11
  warnings.warn('covariance of constraints does not have full '
c:\Users\czk481\Anaconda3\lib\site-packages\statsmodels\base\model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


In [ ]:
Shadow_value = data[['t','idx']]
#data_test['earbubgs'] = pd.qcut(data['log_wealth'], 10, labels = False) 
cat = ['log_earnings_w', 'log_earnings_m','log_earnings_w_l', 'log_earnings_m_l', 'inc_share_w', 'inc_share_w_l','log_wealth', 'log_wealth_l']


#NB SKAL EVT VÆRE FORWARD I STEDET 
for i in cat:
    Shadow_value[i] = pd.qcut(data[i], 10, labels = False, duplicates='raise') 

Shadow_value['earnings_w_m'] = Shadow_value['log_earnings_w'].astype('str') + '_' + Shadow_value['log_earnings_m'].astype('str')
Shadow_value['earnings_w_m_l'] = Shadow_value['log_earnings_w_l'].astype('str') + '_' + Shadow_value['log_earnings_m_l'].astype('str')
Shadow_value['inc_share_n_l'] = Shadow_value['inc_share_w_l'].astype('str') + '_' + Shadow_value['inc_share_w'].astype('str')
Shadow_value['wealth_n_l'] = Shadow_value['log_wealth'].astype('str') + '_' + Shadow_value['log_wealth_l'].astype('str')

Shadow_value = pd.get_dummies(Shadow_value, columns=['earnings_w_m', 'earnings_w_m_l', 'inc_share_n_l','wealth_n_l' ], dtype = float)


#Drop if less thatn two
Shadow_value = Shadow_value.loc[:,(Shadow_value.sum()>2 )]
Shadow_value = Shadow_value.drop(columns = ['t','idx','log_earnings_w', 'log_earnings_m','log_earnings_w_l', 'log_earnings_m_l', 'inc_share_w', 'inc_share_w_l','log_wealth', 'log_wealth_l'])



Shadow_value.info() 
Shadow_value.sum().sort_values()


C:\Users\czk481\AppData\Local\Temp\2\ipykernel_11024\2539133232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shadow_value[i] = pd.qcut(data[i], 10, labels = False, duplicates='raise')
C:\Users\czk481\AppData\Local\Temp\2\ipykernel_11024\2539133232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shadow_value[i] = pd.qcut(data[i], 10, labels = False, duplicates='raise')
C:\Users\czk481\AppData\Local\Temp\2\ipykernel_11024\2539133232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a co

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38504 entries, 0 to 38503
Columns: 229 entries, earnings_w_m_0_0 to wealth_n_l_9_9
dtypes: float64(229)
memory usage: 67.3 MB


earnings_w_m_l_3_1       3.0
wealth_n_l_4_8           3.0
inc_share_n_l_1_7        3.0
wealth_n_l_5_9           4.0
wealth_n_l_6_2           4.0
                       ...  
wealth_n_l_7_7        2724.0
wealth_n_l_0_0        3113.0
wealth_n_l_1_1        3137.0
wealth_n_l_8_8        3337.0
wealth_n_l_9_9        3397.0
Length: 229, dtype: float64

# Estimate the model

In [ ]:
#TODO MANDAG!!
#- create variable lav variable du skal bruge
# - select sample funktion
# lav estimerer omega funktion
# lav main funktion
# lav main funktion med kompliceret skyggeværdi


do_estimate_omega = True
model.sim.init_love[:] = 0.2
model.sim.init_A[:] = 2.0
np.random.seed(model.par.seed)

data = Estimate.create_data(model,start_p = 1, end_p = models-par.T-1)
if do_estimate_omega:
    data = Estimate.aux_est(data, print_reg = True)

data_reg, wald_FC, Wald_NC = Estimate.main_est(data, print_reg= True)

In [ ]:
#Save the data 
#model = models['model LC, NO_HK ']
#model.sim.init_love[:] =0.1
#model.sim.init_A[:] = 6.0
#np.random.seed(model.par.seed)
#data1 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_LC_NO_HK2')


model = models['model LC, HK ']
model.sim.init_love[:] =0.3
model.sim.init_A[:] = 6.0
np.random.seed(model.par.seed)
data2 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_LC_HK')

#model = models['model NC, NO_HK ']
#model.sim.init_love[:] =0.1
#model.sim.init_A[:] = 6.0
#np.random.seed(model.par.seed)
#data1 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_NC_NO_HK2')


model = models['model NC, HK ']
model.sim.init_love[:] =0.3
model.sim.init_A[:] = 6.0
np.random.seed(model.par.seed)
data2 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_NC_HK')

#Save the data 
#model = models['model FC, NO_HK ']
#model.sim.init_love[:] =0.1
#model.sim.init_A[:] = 6.0
#np.random.seed(model.par.seed)
#data1 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_FC_NO_HK2')


model = models['model FC, HK ']
model.sim.init_love[:] =0.3
model.sim.init_A[:] = 6.0
np.random.seed(model.par.seed)
data2 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_FC_HK')


In [ ]:

#Estimate the model one time
data_reg = Estimate.aux_est(data2,print_reg=True)
data_reg, Wald_FC, Wald_NC = Estimate.main_est(data_reg,print_reg=True) 